# Поиск токсичных комментариев на сайте

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 


**План выполнеия работы**

1. Загрузить и подготовить данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
display(comments.head(5))

print('-----------------------------------------------------------------')
comments.info()

print('-----------------------------------------------------------------')

print('Дубликатов -', comments.duplicated().sum())

print('-----------------------------------------------------------------')

print('Пропусков:')
display(comments.isna().sum())

print('-----------------------------------------------------------------')

print('Соотношение в целевом признаке:')
display(comments.toxic.value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


-----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
-----------------------------------------------------------------
Дубликатов - 0
-----------------------------------------------------------------
Пропусков:


Unnamed: 0    0
text          0
toxic         0
dtype: int64

-----------------------------------------------------------------
Соотношение в целевом признаке:


0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Выводы:

- В таблице 159 292 объектов. 
- Пропусков нет, явных дубликатов нет. 
- Тексты комментариев на английском, есть лишние знаки типа "\nMore\n 
- В целевом признаке 90% объектов отрицательного класса, то есть в дальнейшем нужно будет учесть это 

Необходимо избавиться от столбца Unnamed, так как он фактически дублирует индексы  

In [4]:
#удаляю столбец Unnamed:
comments = comments.drop(['Unnamed: 0'], axis=1)

In [5]:
#функция очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [6]:
%%time
#очищаю тексты постов:
comments['text'] = comments['text'].apply(clear_text)

CPU times: user 4.19 s, sys: 23.3 ms, total: 4.22 s
Wall time: 4.22 s


In [7]:
comments.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [8]:
#функция РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
%%time

#леммализирую тексты постов:
comments['text'] = comments['text'].apply(lemm_text) 

CPU times: user 17min 6s, sys: 1min 25s, total: 18min 31s
Wall time: 18min 31s


In [11]:
comments.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [12]:
#разделим выборки в соотношении 80/10/10:
features = comments.drop(['toxic'], axis=1) 
target = comments.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=12345)

In [13]:
#смотрим размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [14]:
#смотрю соотношение 1/0 в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

Доля значений 1 в тренировочной выборке: 0.10166911239631807


#уменьшаю кол-во 0 в выборках train:

comments_train = comments.iloc[target_train.index]
target_train_0 = comments_train[comments_train['toxic'] == 0]['toxic']
target_train_1 = comments_train[comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)

### Выводы:
    
- Удален ненужный столбец
- Очищены тексты комментариев от ненужных знаков
- Тексты леммализированы
- Убраны стоп-слова
- Сбалансированы данные в целевом признаке

## Обучение

#### LogisticRegression

In [15]:
features_train = features_train.text

In [16]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])} 
                                                  
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

F1 логистической регрессии = 0.76
при параметрах {'lr__C': 5, 'lr__class_weight': 'balanced', 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'lbfgs'}

CPU times: user 14min 17s, sys: 9min 17s, total: 23min 35s
Wall time: 23min 35s


In [17]:
#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
print()

F1 логистической регрессии на валидации = 0.76



#### DecisionTreeClassifier

In [18]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), 
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))
print()

F1 дерева решений = 0.63
при параметрах {'dtc__class_weight': 'balanced', 'dtc__max_depth': 24, 'dtc__random_state': 12345}

F1 дерева решений на валидации = 0.63

CPU times: user 12min 25s, sys: 8.31 s, total: 12min 34s
Wall time: 12min 34s


### CatBoostClassifier

In [19]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} 

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
print()

F1 CatBoostClassifier = 0.74
при параметрах {'cbc__class_weights': (1, 1), 'cbc__iterations': 200, 'cbc__verbose': False}

F1 CatBoostClassifier на валидации = 0.76

CPU times: user 38min 43s, sys: 9min 36s, total: 48min 20s
Wall time: 48min 27s


#### RandomForestClassifier

In [20]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([12345]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('F1 случайного леса =', round(rfc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации =', round(rfc_valid_f1,2))
print()

F1 случайного леса = 0.39
при параметрах {'rfc__class_weight': 'balanced', 'rfc__criterion': 'entropy', 'rfc__max_depth': 6, 'rfc__n_estimators': 24, 'rfc__random_state': 12345}

F1 случайного леса на валидации = 0.28

CPU times: user 51min 20s, sys: 26.4 s, total: 51min 46s
Wall time: 51min 48s


#### SGDClassifier

In [21]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("clf", SGDClassifier())])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5),
              'clf__random_state': ([12345]),
              'clf__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
sgdc_train_f1 = max(mts)

print('F1 SGDClassifier =', round(sgdc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

#валидация:
predictions_valid = gscv.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 SGDClassifier на валидации =', round(sgdc_valid_f1,2))
print()

F1 SGDClassifier = 0.75
при параметрах {'clf__class_weight': 'balanced', 'clf__eta0': 0.1, 'clf__learning_rate': 'adaptive', 'clf__loss': 'modified_huber', 'clf__random_state': 12345}

F1 SGDClassifier на валидации = 0.75

CPU times: user 14min 50s, sys: 18.6 s, total: 15min 8s
Wall time: 15min 9s


## Выбор модели для тестирования

In [22]:
#Cводная таблица по показателям F1, времени обучения модели и времени предсказания модели:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier',
         'SGDClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        rfc_valid_f1,
                                        sgdc_valid_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на валидационной выборке', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке
LogisticRegression,0.756144,0.764114
CatBoostClassifier,0.736849,0.759040
SGDClassifier,0.745703,0.746437
DecisionTreeClassifier,0.626301,0.631981
RandomForestClassifier,0.394852,0.280101


#### Наилучшей моделью является LogisticRegression, которая на валидационной выборке показала F1 = 0.76. Применим её для тестирования

In [23]:
#тестирование лучшей модели:
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('Финальный F1 логистической регрессии =', round(lr_test_f1,2))

Финальный F1 логистической регрессии = 0.75


### Выводы:

- Данные загружены и проведена их предобработка - удаление лишних данных, очистка текстов, лемматизация.

- Обучены 5 моделей с разными гиперпараметрами и выборками и проведена проверка их на тестовой выборке.

- Выбрана лучшая модель по показателю F1. Худшими показателями среди моделей обладают RandomForestClassifier и DecisionTreeClassifier, так как дали наименьшее F1.

- Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.75. 